# API Mapping version 1

Description
- Embedding contains both code, intent, methodname, classname works well with capturing one to many mapping in language specific features like overloading and destructors.
- On few examples not precise with method parameter and signature potential chance of losing mapping with language specific features like overloading; java code - class Complex {
double real, imag;
Complex(double r, double i) { real = r; imag = i; }
public Complex add(Complex other) {
return new Complex(real + other.real, imag + other.imag);
}
} resposne - The methods and mapping are, ['{\n"match": false,\n"java_methods_used": ["add"],\n"api_flow": ["add"]}]
- Failing in api granularity match like mean, Geomentric mean, harmonic mean - which would be broken down to sum and divison
- Uses the embedding intfloat/multilingual-e5-base


In [1]:
from langchain_core.documents import Document
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_groq import ChatGroq
import requests, json, re
import os
import pandas as pd

In [2]:
def create_api_mapping(cpp_code, java_methods_info):
     
     # Prepare the prompt
        prompt = f"""
You are an expert software engineer with good knowledge in C++ and Java .
You are given the following C++ method:
{cpp_code}


And the following top {len(java_methods_info)} candidate Java methods:
{java_methods_info}


Task:
1. Determine if these Java methods can together implement the exact behavior of the C++ method.
2. If yes, specify only those APIs are used, their relationship (AND/OR) between the java methods, and the flow or the order which methods follows which methods
3. If not, say "No exact match".
4. The mapping of java method and cpp code whether its one to one or one cpp to many java
5. And give the retrieved intent of the java methods that either standalone or together implement the exact behavior of the C++ method
Respond in structured JSON:
{{
"match": true/false,
"java_methods_used": [list of method names],
"api_flow": [list of APIs and relationships]
"Mapping" : one to one cpp java mapping or one cpp to many java method mapping
"Intent": intent of all selected java methods that implement exact behaviour of C++ methods
}}
"""

        api_key = os.environ["API_KEY"]
        response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": "qwen/qwen-2.5-coder-32b-instruct",
            "messages": [
                {"role": "system", "content": "You are a helpful expert software analyst."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens":1024,
            "temperature": 0.2,
            
        })
        )
        result = response.json()
        print(result)
        return result["choices"][0]["message"]["content"]

In [3]:
def intent_generator(code):

        # Prepare the prompt
        prompt = f"""
        Analyze the following C++ or java source code and write a very very short intent and how it performs that intent of the method.
        just two sentences. it should also include how it performs as operation
        Excepted Output:
        "Intent: What is does, Operation: How it does"

        C++ or java Source Code:
        {code}
        """

        # Build the request payload for a chat-based model (like Mistral or Falcon via OpenRouter)
        data = {
            "model": "mistralai/mixtral-8x7b-instruct",  # or another model available on OpenRouter
            "messages": [
                {"role": "system", "content": "You are a helpful expert software analyst."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens": 1024,
            "temperature": 0.2
        }

        headers = {
            "Authorization": f"Bearer {os.environ['API_KEY']}",
            "Content-Type": "application/json"
        }

        response = requests.post('https://openrouter.ai/api/v1/chat/completions', json=data, headers=headers)

        if response.status_code == 200:
            result = response.json()
            # Extract generated text from the response structure
            description = result["choices"][0]["message"]["content"]
            return(description)
        else:
            return(f"Error: {response.status_code} - {response.text}")

In [ ]:
code = """public <T extends Number> double add(T a, T b) {
return a.doubleValue() + b.doubleValue();
}"""
result = intent_generator(code)
result

In [4]:
code_list = [{'code': 'public void printMessage() {\nSystem.out.println("Hello, World!");\n}',
  'methodname': 'printMessage','classname':'',
  'intent': ' This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.'},
 {'code': 'public int maxVal(int a, int b) {\nreturn (a > b) ? a : b;\n}',
  'methodname': 'maxVal','classname':'',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`. The method returns the maximum of the two input values by using a conditional operator. If `a` is greater than `b`, the method returns `a`, otherwise it returns `b`.'},
 {'code': 'public int add(int a) {\nreturn add(a, 10);\n}',
  'methodname': 'add','classname':'',
  'intent': ' This Java method, named "add", takes an integer "a" as an argument and calls the "add" method with arguments "a" and 10. The return type is an integer. The specific implementation of the add method is not provided, but it appears to perform some form of addition.'},
 {'code': 'public int add(int a, int b) {\nreturn a + b;\n}',
  'methodname': 'add','classname':'',
  'intent': ' This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the \'+\' operator.'},
 {'code': 'public double multiply(double a, double b) {\nreturn a * b;\n}',
  'methodname': 'multiply','classname':'',
  'intent': " This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase."},
 {'code': 'public int multiply(int a, int b) {\nreturn a * b;\n}',
  'methodname': 'multiply','classname':'',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.'},
 {'code': 'public void increment(MutableInt x) {\nx.value++;\n}',
  'methodname': 'increment','classname':'MutableInt',
  'intent': ' This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.'},
 {'code': 'class MutableInt {\npublic int value;\npublic MutableInt(int v) { value = v; }\n}',
  'methodname': '','classname':'MutableInt',
  'intent': ' The `MutableInt` class in C++ or Java is a simple container for an integer value. It provides a constructor to initialize the value and allows direct access to the value, making it mutable.'},
 {'code': 'class MathUtils {\npublic static int square(int x) {\nreturn x * x;\n}\n}',
  'methodname': 'square','classname':'MathUtils',
  'intent': ' The `square` method in the `MathUtils` class takes an integer `x` as an argument and returns the result of multiplying `x` by itself, which is the square of the input. This method is a simple utility for performing a square operation in a mathematical context.'},
 {'code': '\nclass Animal {\npublic void speak() {\nSystem.out.println("Animal sound");\n}\n}\n ',
  'methodname': 'speak','classname':'Animal',
  'intent': ' The `speak` method in the `Animal` class in the provided C++ or Java source code is a public method that prints the message "Animal sound" to the console. This method provides a simple way to make an abstract representation of an animal produce a sound, although it does not provide any specific sound for any particular animal species.'},
 {'code': ' \nclass Dog extends Animal {\n@Override\npublic void speak() {\nSystem.out.println("Woof!");\n}\n}\n',
  'methodname': 'speak','classname':'Animal, Dog',
  'intent': ' This is a Java method that overrides the speak() method in the Dog class, which is a subclass of the Animal class. When this method is called, it prints the string "Woof!" to the console, representing the sound that a dog makes.'},
 {'code': '\n public <T extends Number> double add(T a, T b) {\nreturn a.doubleValue() + b.doubleValue();\n}\n\n ',
  'methodname': 'add','classname':'Number',
  'intent': ' This method, with a generic type T that extends the Number class, is implemented in either Java or a compatible language. Its purpose is to add together two numeric values of the same type, returning the result as a double precision floating-point number.'},
 {'code': '\nclass FileHandler {\n// No automatic destructor; must use explicit method\npublic void close() {\nSystem.out.println("Cleaning up resources.");\n}\n@Override\nprotected void finalize() throws Throwable {\n// Called by GC, but not reliable\nclose();\n}\n}\n  ',
  'methodname': 'close','classname':'FileHandler',
  'intent': ' The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup. The `finalize()` method is overridden to call `close()`, but it is not reliable as it is dependent on the garbage collector.'},
 {'code': '\nclass Complex {\ndouble real, imag;\nComplex(double r, double i) { real = r; imag = i; }\npublic Complex add(Complex other) {\nreturn new Complex(real + other.real, imag + other.imag);\n}\n}\n ',
  'methodname': '','classname':'Complex',
  'intent': ' This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to represent complex numbers. The `add` method takes a `Complex` number as an argument and returns a new `Complex` number representing the sum of the current and argument complex numbers.'},
  {'code': """public static double divide(double numerator, double denominator) {
        if (denominator == 0) {
            throw new ArithmeticException("Cannot divide by zero");
        }
        return numerator / denominator;
    }
""",
  'methodname': 'divide','classname':'','intent':'The function takes tow numbers and divides it'},
  {'code': """public class MathUtils {

    // Method to return the square root of a number
    public static double squareRoot(double number) {
        if (number < 0) {
            throw new IllegalArgumentException("Cannot compute square root of a negative number.");
        }
        return Math.sqrt(number);
    }
""",
  'methodname': 'squareRoot','classname':'MathUtils','intent':'The function takes a number and gives the square root of it'}]

In [ ]:
for code in code_list:
    code['intent'] = intent_generator(code['code'])

In [5]:
def create_documents_from_chunks(chunk_list):
    list_of_documents = []
    for chunk in chunk_list:
        document = Document(
            page_content=f"code: {chunk['code']},'intent': {chunk['intent']},methods: {chunk['methodname']},classname: {chunk['classname']}",
        )
        list_of_documents.append(document)
    return list_of_documents
java_method_documents = create_documents_from_chunks(code_list)

In [6]:
for i in java_method_documents:
    print(i.page_content)

code: public void printMessage() {
System.out.println("Hello, World!");
},'intent':  This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.,methods: printMessage,classname: 
code: public int maxVal(int a, int b) {
return (a > b) ? a : b;
},'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`. The method returns the maximum of the two input values by using a conditional operator. If `a` is greater than `b`, the method returns `a`, otherwise it returns `b`.,methods: maxVal,classname: 
code: public int add(int a) {
return add(a, 10);
},'intent':  This Java method, named "add", takes an integer "a" as an argument and calls the "add" method with arguments "a" and 10. The return type is an integer. The specific implementation of the add method is not provided, but it appears to perform some form of addition.,methods: add,classname: 
code: public in

In [7]:
embedding = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-base",
            model_kwargs={"device": "cpu"})
vector_store = FAISS(
            embedding_function=embedding,
            index=faiss.IndexFlatL2(len(embedding.embed_query("hello world"))),
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
        )

C:\Users\soham\AppData\Local\Temp\ipykernel_7048\2807852418.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings(
c:\c\Users\Hari\VISTAR_PROJECT\SM7to8_api_knowledge_mapping\myenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [8]:
vector_store.add_documents(documents=java_method_documents)

['d759b033-9ab4-4f33-8919-94e2b2558ef8',
 '922f200b-52e9-4a7a-a255-253bccdc5e40',
 '6d7da48d-7980-4cc0-9a9f-c6020d1b22c8',
 '3660cc4d-c274-4332-9dd1-7535fbd804fc',
 '6470414a-106f-4eb6-a297-d5f4708ebf16',
 '28b88a41-c01b-49bf-8e88-c4ee732b5e62',
 '1e34f7a4-8749-43bb-9ed8-008579edadb0',
 'cb8651e6-03bf-47bb-abc9-432bbfbc1c6b',
 '3ca942cf-d3fb-4a0d-99f9-ba89328dfe86',
 'd4071fc8-0f95-43b7-a48c-df96d1c28956',
 '38e77b7f-e637-4500-a6c4-8ae9e54766cf',
 'fde95748-be87-45ea-962c-e383a75fdaff',
 'e53713b4-b125-4c43-b699-35008230884e',
 'a1f65375-2abf-4bf2-9973-02cba1806457',
 '9136e51b-abc9-4c35-b57c-9614b0301f5a',
 'ed8ad718-bd48-4cab-bc91-7ff3828f4ce3']

In [ ]:
list_of_dict_of_mapping = []

In [10]:
# Example 10
code = """
 class Complex {
public:
double real, imag;
Complex(double r, double i): real(r), imag(i) {}
Complex operator+(const Complex& other) {
return Complex(real + other.real, imag + other.imag);
}
};
"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])
# list_of_dict_of_mapping.append(my_dict)

* code: 
class Complex {
double real, imag;
Complex(double r, double i) { real = r; imag = i; }
public Complex add(Complex other) {
return new Complex(real + other.real, imag + other.imag);
}
}
 ,'intent':  This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to represent complex numbers. The `add` method takes a `Complex` number as an argument and returns a new `Complex` number representing the sum of the current and argument complex numbers.,methods: ,classname: Complex
* code: public int add(int a, int b) {
return a + b;
},'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* code: 
 public <T extends Number> double add(T a, T b) {
return a.doubleValue() + b.doubleValue();
}

 ,'intent':  This method, with a generic type T that extends the Number class, is implemented in either Java or a compa

In [12]:
# example 3
code = """
 C++
int add(int a, int b = 10) {
return a + b;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])
my_dict
# list_of_dict_of_mapping.append(my_dict)

 Intent: This method performs an addition operation, returning the sum of two integers.
Operation: It takes an integer 'a' as a parameter and an optional integer 'b' with a default value of 10, and returns the sum of the two numbers by adding them together using the '+' operator.
* code: public int add(int a, int b) {
return a + b;
},'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* code: public int add(int a) {
return add(a, 10);
},'intent':  This Java method, named "add", takes an integer "a" as an argument and calls the "add" method with arguments "a" and 10. The return type is an integer. The specific implementation of the add method is not provided, but it appears to perform some form of addition.,methods: add,classname: 
* code: public int multiply(int a, int b) {
return a * b;
},'intent':  This is a simple method in Java th

{'match': True,
 'java_methods_used': ['add(int a)', 'add(int a, int b)'],
 'api_flow': ['add(int a) -> add(int a, int b)'],
 'Mapping': 'one cpp to many java method mapping',
 'Intent': 'This Java method, named "add", takes an integer "a" as an argument and optionally another integer "b" (defaulting to 10 if not provided). The method returns the sum of these integers, calculated by using the \'+\' operator.'}

In [13]:
# Example 5
code = """
void increment(int &x) {
x++;
}

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])
my_dict
# list_of_dict_of_mapping.append(my_dict)

* code: public void increment(MutableInt x) {
x.value++;
},'intent':  This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.,methods: increment,classname: MutableInt
* code: class MutableInt {
public int value;
public MutableInt(int v) { value = v; }
},'intent':  The `MutableInt` class in C++ or Java is a simple container for an integer value. It provides a constructor to initialize the value and allows direct access to the value, making it mutable.,methods: ,classname: MutableInt
* code: public int add(int a, int b) {
return a + b;
},'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* code: public int add(int a) {
return add(a, 10);
},'intent':  This Java method,

{'match': True,
 'java_methods_used': ['increment'],
 'api_flow': ['increment(MutableInt x)'],
 'Mapping': 'one cpp to one java method mapping',
 'Intent': 'This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.'}

In [14]:
# example 7
code = """
class FileHandler {
public:
~FileHandler() {
// Automatically called when object goes out of scope
std::cout << "Cleaning up resources." << std::endl;
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])
# list_of_dict_of_mapping.append(my_dict)
my_dict

* code: 
class FileHandler {
// No automatic destructor; must use explicit method
public void close() {
System.out.println("Cleaning up resources.");
}
@Override
protected void finalize() throws Throwable {
// Called by GC, but not reliable
close();
}
}
  ,'intent':  The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup. The `finalize()` method is overridden to call `close()`, but it is not reliable as it is dependent on the garbage collector.,methods: close,classname: FileHandler
* code: public void printMessage() {
System.out.println("Hello, World!");
},'intent':  This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.,methods: printMessage,classname: 
* code: class MathUtils {
public static int square(int x) {
return x * x;
}
}

{'match': False,
 'java_methods_used': ['close'],
 'api_flow': ['close()'],
 'Mapping': 'one cpp to many java method mapping',
 'Intent': 'The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup.'}

In [16]:
# example 7
code = """
class Animal {
public:
virtual void speak() {
std::cout << "Animal sound" << std::endl;
}
};
class Dog : public Animal {
public:
void speak() override {
std::cout << "Woof!" << std::endl;
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])
# list_of_dict_of_mapping.append(my_dict)
my_dict

* code:  
class Dog extends Animal {
@Override
public void speak() {
System.out.println("Woof!");
}
}
,'intent':  This is a Java method that overrides the speak() method in the Dog class, which is a subclass of the Animal class. When this method is called, it prints the string "Woof!" to the console, representing the sound that a dog makes.,methods: speak,classname: Animal, Dog
* code: 
class Animal {
public void speak() {
System.out.println("Animal sound");
}
}
 ,'intent':  The `speak` method in the `Animal` class in the provided C++ or Java source code is a public method that prints the message "Animal sound" to the console. This method provides a simple way to make an abstract representation of an animal produce a sound, although it does not provide any specific sound for any particular animal species.,methods: speak,classname: Animal
* code: public void printMessage() {
System.out.println("Hello, World!");
},'intent':  This is a Java method named "printMessage" which prints the str

{'match': True,
 'java_methods_used': ['speak'],
 'api_flow': ['Animal.speak()', 'Dog.speak()'],
 'Mapping': 'one cpp to many java method mapping',
 'Intent': "The `speak` method in the `Animal` class provides a generic implementation that prints 'Animal sound' to the console. The `speak` method in the `Dog` class overrides this method to provide a specific implementation that prints 'Woof!' to the console, representing the sound that a dog makes."}

In [17]:
# Example Mean:
code = """
 int mean(int a, int b) {
    return (a + b) / 2.0;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculate the average of two integers, Operation: By adding the two integers, dividing the sum by 2.0 to ensure floating point division.
* code: public int add(int a, int b) {
return a + b;
},'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* code: public double multiply(double a, double b) {
return a * b;
},'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* code: 
 public <T extends Number> double add(T a, T b) {
return a.doubleValue() + b.doubleValue();
}

 ,'intent':  This method, with a generic type T that exten

In [ ]:
df = pd.DataFrame(list_of_dict_of_mapping)
print(df)

# API MAPPING VERSION 2

Description

- At times the methods captured are not clear but the flow is good through this we can get that information but keeping both methods and flow working representing idependenlty can improve the results. code - mean of two number - response - The methods and mapping are, ['{\n  "match": true,\n  "java_methods_used": ["add"],\n  "api_flow": ["add(a, b) / 2"]}]
- failed at caputring the language feature mismatch. ["add"],\n  "api_flow": ["add(Complex other) - AND"],\n  "Mapping": "one-to-one"
- uses the embedding intfloat/multilingual-e5-base

In [2]:
def create_api_mapping(cpp_code, java_methods_info):
     
     # Prepare the prompt
        prompt = f"""
You are an expert software engineer with deep knowledge of C++ and Java.

C++ method:
{cpp_code}

Candidate Java methods (with intents):
{java_methods_info}

Task:
1. Analyze the C++ method's behavior (not just the code, but its intent and required operations).
2. Identify which Java methods individually OR in combination can reproduce this behavior.
   - If no single method exists, check if multiple retrieved methods can be composed (e.g., method1() + method2()).
3. For each potential match:
   - List the methods used.
   - Describe the relationship: AND (must be used together) or OR (can be used interchangeably).
   - Provide the execution flow/sequence.
4. If no methods (individually or combined) can implement the behavior, say "No exact match".
5. Output the result as structured JSON:
{{
  "match": true/false,
  "java_methods_used": [list of method names],
  "api_flow": ["sequence of methods with relationships"],
  "Mapping": "one-to-one" or "one-to-many",
  "Intent": "Summarized intent of the selected methods that together replicate the C++ behavior"
}}
"""

        api_key = os.environ["API_KEY"]
        response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": "qwen/qwen-2.5-coder-32b-instruct",
            "messages": [
                {"role": "system", "content": "You are a helpful expert software analyst."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens":1024,
            "temperature": 0.2,
            
        })
        )
        result = response.json()
        print(result)
        return result["choices"][0]["message"]["content"]

In [3]:
def intent_generator(code):

        # Prepare the prompt
        prompt = f"""
        Analyze the following C++ or java source code and write a very very short intent and how it performs that intent of the method.
        just two sentences. it should also include how it performs as operation
        Excepted Output:
        "Intent: What is does, Operation: How it does"

        C++ or java Source Code:
        {code}
        """

        # Build the request payload for a chat-based model (like Mistral or Falcon via OpenRouter)
        data = {
            "model": "mistralai/mixtral-8x7b-instruct",  # or another model available on OpenRouter
            "messages": [
                {"role": "system", "content": "You are a helpful expert software analyst."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens": 1024,
            "temperature": 0.2
        }

        headers = {
            "Authorization": f"Bearer {os.environ['API_KEY']}",
            "Content-Type": "application/json"
        }

        response = requests.post('https://openrouter.ai/api/v1/chat/completions', json=data, headers=headers)

        if response.status_code == 200:
            result = response.json()
            # Extract generated text from the response structure
            description = result["choices"][0]["message"]["content"]
            return(description)
        else:
            return(f"Error: {response.status_code} - {response.text}")

In [ ]:
code_list = [{'methodname': 'printMessage','classname':'',
  'intent': ' This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.'},
 {'methodname': 'maxVal','classname':'',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`. The method returns the maximum of the two input values by using a conditional operator. If `a` is greater than `b`, the method returns `a`, otherwise it returns `b`.'},
 {'methodname': 'add','classname':'',
  'intent': ' This Java method, named "add", takes an integer "a" as an argument and calls the "add" method with arguments "a" and 10. The return type is an integer. The specific implementation of the add method is not provided, but it appears to perform some form of addition.'},
 {'methodname': 'add','classname':'',
  'intent': ' This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the \'+\' operator.'},
 {'methodname': 'multiply','classname':'',
  'intent': " This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase."},
 {'methodname': 'multiply','classname':'',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.'},
 {'methodname': 'increment','classname':'MutableInt',
  'intent': ' This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.'},
 {'methodname': '','classname':'MutableInt',
  'intent': ' The `MutableInt` class in C++ or Java is a simple container for an integer value. It provides a constructor to initialize the value and allows direct access to the value, making it mutable.'},
 {'methodname': 'square','classname':'MathUtils',
  'intent': ' The `square` method in the `MathUtils` class takes an integer `x` as an argument and returns the result of multiplying `x` by itself, which is the square of the input. This method is a simple utility for performing a square operation in a mathematical context.'},
 {'methodname': 'speak','classname':'Animal',
  'intent': ' The `speak` method in the `Animal` class in the provided C++ or Java source code is a public method that prints the message "Animal sound" to the console. This method provides a simple way to make an abstract representation of an animal produce a sound, although it does not provide any specific sound for any particular animal species.'},
 {'methodname': 'speak','classname':'Animal, Dog',
  'intent': ' This is a Java method that overrides the speak() method in the Dog class, which is a subclass of the Animal class. When this method is called, it prints the string "Woof!" to the console, representing the sound that a dog makes.'},
 {'methodname': 'add','classname':'Number',
  'intent': ' This method, with a generic type T that extends the Number class, is implemented in either Java or a compatible language. Its purpose is to add together two numeric values of the same type, returning the result as a double precision floating-point number.'},
 {'methodname': 'close','classname':'FileHandler',
  'intent': ' The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup. The `finalize()` method is overridden to call `close()`, but it is not reliable as it is dependent on the garbage collector.'},
 {'methodname': '','classname':'Complex',
  'intent': ' This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to represent complex numbers. The `add` method takes a `Complex` number as an argument and returns a new `Complex` number representing the sum of the current and argument complex numbers.'},
  {'methodname': 'divide','classname':'','intent':'The function takes two numbers and divides it'},
  {'methodname': 'squareRoot','classname':'MathUtils','intent':'The function takes a number and gives the square root of it'}]

In [5]:
def create_documents_from_intents(chunk_list):
    list_of_documents = []
    for chunk in chunk_list:
        document = Document(
            page_content=f"'intent': {chunk['intent']},methods: {chunk['methodname']},classname: {chunk['classname']}",
            metadata = {'methods': chunk['methodname'],'classname': chunk['classname']}
        )
        list_of_documents.append(document)
    return list_of_documents
only_intent_document = create_documents_from_intents(code_list)

In [6]:
embedding = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-base",
            model_kwargs={"device": "cpu"})
vector_store1 = FAISS(
            embedding_function=embedding,
            index=faiss.IndexFlatL2(len(embedding.embed_query("hello world"))),
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
        )
vector_store1.add_documents(only_intent_document)

C:\Users\soham\AppData\Local\Temp\ipykernel_15284\4282650126.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings(
c:\c\Users\Hari\VISTAR_PROJECT\SM7to8_api_knowledge_mapping\myenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


['ffc73d95-46d2-4ecc-9471-94bb55d0c543',
 '3bd7630a-3623-4e82-a9e6-ae9609d5b41b',
 '095523cc-bf4e-4587-82bb-133ff19d9c22',
 'd41011b7-27be-4a7d-84e8-c37459967e8d',
 'f3f5bf41-8339-437e-bfc6-dda5d2be7c5b',
 '26a691de-c725-4512-a9fe-9eb83b5e60b4',
 'f4bc2202-e5b6-42c3-9e90-99e9d1d397e3',
 '83a389d3-4f62-4ffe-89df-452aa1a46996',
 'f12513ac-0d93-4566-beeb-bba5c8491af7',
 '107c3d8b-6a72-46df-92e9-50c3fab89eef',
 '44b37920-1fd9-4616-86b1-b67fbae7e8c5',
 'cdec4eb7-e37c-4427-bdf6-3514b5753954',
 '4b68c30b-8dbc-4593-a260-f0dc2ff37173',
 'fa4cf492-2985-40ef-85d8-78aa372bd8c0',
 '08938ece-c577-46d1-93d9-79dda9598dbd',
 '9abcd6da-8e01-4ed0-be87-42f8f45ab808']

In [17]:
# Example Mean:
code = """
 int mean(int a, int b) {
    return (a + b) / 2.0;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculates the mean (average) of two integers, Operation: By adding the two integers, dividing the sum by 2.0, and returning the result as a floating point number.
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent': The function takes a number and gives the

In [14]:
# Example Root Mean:
code = """
 int rootMeanSquare(int a, int b) {
    return std::sqrt((a * a + b * b) / 2.0);
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculates the root mean square of two integers, providing a measure of their average magnitude.
Operation: Performs integer multiplication, addition, and division, then obtains the square root of the result using a floating-point number.
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent':  This is a Java method named "add" that takes two integer par

In [10]:
# Example geomentric Mean:
code = """
 double geometricMean(double a, double b) {
    return std::sqrt(a * b);
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculate the geometric mean of two numbers, Operation: It multiplies the numbers together, then takes the square root of the result.
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: 
{'id': 'gen-1756457037-KX3AjObKIl2jCw0VfH2z', 'provider': 'Nebius', 'model': 'qwen/qwen-2.5-

In [11]:
# Example harmonic mean
code = """
 double harmonicMean(double a, double b) {
    return (2.0 * a * b) / (a + b);
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculates the harmonic mean of two numbers, Operation: Performs this calculation by invoking the formula (2 * a * b) / (a + b) using given numbers a and b.
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent': The function takes a number and gives the square

In [15]:
# Example variance
code = """
double variance(double a, double b) {
    double mean = (a + b) / 2.0;
    return ((a - mean) * (a - mean) + (b - mean) * (b - mean)) / 2.0;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculate the variance of two numbers, Operation: By finding the mean of the two numbers, then calculating the difference of each number from the mean, squaring those differences, averaging the squared differences, and returning the result.
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: 
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multip

In [13]:
# Example Standard Deviation
code = """
 double standardDeviation(double a, double b) {
    double mean = (a + b) / 2.0;
    double variance = ((a - mean) * (a - mean) + (b - mean) * (b - mean)) / 2.0;
    return std::sqrt(variance);
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculates the standard deviation of two numbers, operation: By first finding the mean, then calculating the variance, and finally returning the square root of the variance.
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: 
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
{'id': 'gen-1756457423-PSkjvEBrVOxtMLDfwyvS', 'provi

In [ ]:
# Example Default argument
code = """
 int add(int a, int b = 10) {
return a + b;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: The `add` method returns the sum of two integers.
Operation: It performs addition of the first argument 'a' with the second argument 'b' having a default value of 10.
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* 'intent':  This Java method, named "add", takes an integer "a" as an argument and calls the "add" method with arguments "a" and 10. The return type is an integer. The specific implementation of the add method is not provided, but it appears to perform some form of addition.,methods: add,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent':  This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to repr

In [19]:
# Example Pass by reference

code = """
 void increment(int &x) {
x++;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Increments the value of a passed integer variable, Operation: Performs this operation in-place by directly accessing and modifying the memory location of the variable.
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent':  This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.,methods: increment,classname: MutableInt
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
{'id': 'gen-1756459

In [22]:
# Example Reference

code = """
 void increment(int &x) {
x++;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Increments the value of a given integer, Operation: Performs this increment directly on the integer reference provided as an argument, using the postfix increment operator '++'.
* 'intent':  This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.,methods: increment,classname: MutableInt
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: 
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
{'id': 'g

In [ ]:
# example File Handler example
code = """
class FileHandler {
public:
~FileHandler() {
// Automatically called when object goes out of scope
std::cout << "Cleaning up resources." << std::endl;
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])


* 'intent':  The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup. The `finalize()` method is overridden to call `close()`, but it is not reliable as it is dependent on the garbage collector.,methods: close,classname: FileHandler
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils
* 'intent':  This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.,methods: printMessage,classname: 
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: 
{'id': 'gen-1756460411-5piIzBOICOU2i658bngH', 'provider': 'DeepInfra', 'model': 'qwen/qwen-2.5-coder-32b-instruct', 'object': 'chat.completion', 'created': 1756460411, 'choices': [{'logprobs': None, 'finish_

In [24]:
# example 7
code = """
class Complex {
public:
double real, imag;
Complex(double r, double i): real(r), imag(i) {}
Complex operator+(const Complex& other) {
return Complex(real + other.real, imag + other.imag);
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])


* 'intent':  This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to represent complex numbers. The `add` method takes a `Complex` number as an argument and returns a new `Complex` number representing the sum of the current and argument complex numbers.,methods: ,classname: Complex
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: 
* 'intent':  This method, with a generic type T that extends the Number class, is implemented in either Java or a compatible language. Its purpose is to add together two numeric values of the same type, returning the result as a double precision floating-point number.,methods: add,classname: Number
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result

# API_MAPPING_VERSION_CHAIN_OF_THOUGHT

In [23]:
import os
import requests
import json

def create_api_mapping(cpp_code, java_methods_info):
      
    # Prepare the prompt
    prompt = f"""You are an expert software engineer with deep expertise in both C++ and Java.

    ### Guidelines

Lets think step by step:

Step 1 — Extract behaviour & types:
  1.1 Parse the C++ method to extract:
      - operation intent (e.g., increment, add, average)
      - parameter types (including whether a parameter is a reference '&' or pointer '*')
      - return type
      - any C++-specific language features used (detect: operator overloading, default arguments, pass-by-reference, pointers, destructors, copy/move semantics, templates, implicit conversions, RAII).
  1.2 If a language feature is detected, record it in a list `LanguageFeaturesDetected`.

Step 2 — Build type-compatible Java candidate set:
  2.1 From list of java methods, select only those Java methods whose signatures and domain types are TYPE-COMPATIBLE with the C++ types (prefer same domain class names and fully matching types). 
  2.2 If C++ uses a user-defined type (e.g., Complex), exclude Java methods that operate only on unrelated primitives unless an explicit adapter method (e.g., Complex.add(Complex)) exists in the candidates.
  2.3 For pass-by-reference: treat candidate Java methods that accept mutable wrappers (e.g., MutableInt, AtomicInteger, IntHolder, int[]) as type-compatible adapters.

Step 3 — Determine composition & functional equivalence:
  3.1 Check if any single candidate method EXACTLY reproduces the C++ behaviour (same effects, side-effects, types). If yes, it is a functional match.
  3.2 If no single method, search for combinations of candidate methods that can be composed (sequence or combination) to reproduce the behaviour. Composition is allowed only if parameter/return types and intents are compatible.

Step 4 — Language-feature mapping rule (CRITICAL):
  4.1 If `LanguageFeaturesDetected` is non-empty (e.g., pass-by-reference, operator overloading, destructors, default args, pointers, templates), then this is a **Language Feature Translation** and **MUST** be labeled as `"Mapping": "one-to-many"`, **even if only ONE Java adapter method** from the candidate set is used to simulate the feature.  
  4.2 Add `"MappingCategory": "Language Feature Translation"` to the result whenever any language feature is detected.

Step 5 — Failure:
  5.1 If no single method nor any legal composition of provided candidate methods can exactly reproduce the C++ behaviour, output `"match": false` and `"java_methods_used": []` with `"Mapping": "none"` and include `"LanguageFeaturesDetected"` if any.



### Example 1:
C++ method:
int mean(int a, int b) {{
    return (a + b) / 2.0;
}}

Candidate Java methods:
- add(int a, int b)
- divide(double a, double b)
- multiply(double a, double b)
- squareRoot(double x)

Reasoning:
1. This computes the average of two integers.
2. We need to add the two integers, then divide by 2.0.
3. Mapping: one-to-many because two Java methods are used.

JSON:
{{
  "match": true,
  "java_methods_used": ["add(int, int)", "divide(double, double)"],
  "api_flow": ["add(int, int) -> divide(double, double)"],
  "Mapping": "one-to-many",
  "Intent": "Adds two integers and divides the sum by 2.0 to find the mean."
}}

---

### Example 2:
C++ method:
Complex Complex::operator+(const Complex& other) {{
    return Complex(real + other.real, imag + other.imag);
}}

Candidate Java methods:
- Complex.add(Complex other)
- add(int a, int b)
- multiply(double a, double b)

Reasoning:
1. This adds two complex numbers using operator overloading.
2. The Java equivalent is Complex.add(Complex other).
3. Mapping: one-to-many because it is a Language Feature Mappings like operator overloading in C++ translates to many method in Java.

JSON:
{{
  "match": true,
  "java_methods_used": ["Complex.add(Complex)"],
  "api_flow": ["Complex.add(Complex)"],
  "Mapping": "one-to-many",
  "Intent": "Adds two complex numbers just like operator+."
}}

---

### Now your task:
C++ method:
{cpp_code}

Candidate Java methods:
{java_methods_info}

Follow the same reasoning steps and return only the JSON result.
"""

    api_key = os.environ["API_KEY"]
    response = requests.post(
    url="https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    },
    data=json.dumps({
        "model": "qwen/qwen-2.5-coder-32b-instruct",
        "messages": [
            {"role": "system", "content": "You are a helpful expert software analyst."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens":1024,
        "temperature": 0.2,
        
    })
    )
    result = response.json()
    print(result)
    return result["choices"][0]["message"]["content"]

In [3]:
def intent_generator(code):

        # Prepare the prompt
        prompt = f"""
        Analyze the following C++ or java source code and write a very very short intent and how it performs that intent of the method.
        just two sentences. it should also include how it performs as operation
        Excepted Output:
        "Intent: What is does, Operation: How it does"

        C++ or java Source Code:
        {code}
        """

        # Build the request payload for a chat-based model (like Mistral or Falcon via OpenRouter)
        data = {
            "model": "mistralai/mixtral-8x7b-instruct",  # or another model available on OpenRouter
            "messages": [
                {"role": "system", "content": "You are a helpful expert software analyst."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens": 1024,
            "temperature": 0.2
        }

        headers = {
            "Authorization": f"Bearer {os.environ['API_KEY']}",
            "Content-Type": "application/json"
        }

        response = requests.post('https://openrouter.ai/api/v1/chat/completions', json=data, headers=headers)

        if response.status_code == 200:
            result = response.json()
            # Extract generated text from the response structure
            description = result["choices"][0]["message"]["content"]
            return(description)
        else:
            return(f"Error: {response.status_code} - {response.text}")

In [4]:
code_list = [{'code': 'public void printMessage() {\nSystem.out.println("Hello, World!");\n}',
  'methodname': 'printMessage','classname':'','method_signature':'printMessage()',
  'intent': ' This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.'},
 {'code': 'public int maxVal(int a, int b) {\nreturn (a > b) ? a : b;\n}',
  'methodname': 'maxVal','classname':'','method_signature':'maxVal(int , int )',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`. The method returns the maximum of the two input values by using a conditional operator. If `a` is greater than `b`, the method returns `a`, otherwise it returns `b`.'},
 {'code': 'public int add(int a) {\nreturn add(a, 10);\n}',
  'methodname': 'add','classname':'','method_signature':'add(int )',
  'intent': ' This Java method, named "add", takes an integer "a" as an argument and calls the "add" method with arguments "a" and 10. The return type is an integer. The specific implementation of the add method is not provided, but it appears to perform some form of addition.'},
 {'code': 'public int add(int a, int b) {\nreturn a + b;\n}',
  'methodname': 'add','classname':'','method_signature':'add(int , int )',
  'intent': ' This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the \'+\' operator.'},
 {'code': 'public double multiply(double a, double b) {\nreturn a * b;\n}',
  'methodname': 'multiply','classname':'','method_signature':'multiply(double , double )',
  'intent': " This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase."},
 {'code': 'public int multiply(int a, int b) {\nreturn a * b;\n}',
  'methodname': 'multiply','classname':'','method_signature':'multiply(int , int )',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.'},
 {'code': 'public void increment(MutableInt x) {\nx.value++;\n}',
  'methodname': 'increment','classname':'MutableInt','method_signature':'increment(MutableInt )',
  'intent': ' This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.'},
 {'code': 'class MutableInt {\npublic int value;\npublic MutableInt(int v) { value = v; }\n}',
  'methodname': '','classname':'MutableInt','method_signature':'MutableInt(int )',
  'intent': ' The `MutableInt` class in C++ or Java is a simple container for an integer value. It provides a constructor to initialize the value and allows direct access to the value, making it mutable.'},
 {'code': 'class MathUtils {\npublic static int square(int x) {\nreturn x * x;\n}\n}',
  'methodname': 'square','classname':'MathUtils','method_signature':'square(int )',
  'intent': ' The `square` method in the `MathUtils` class takes an integer `x` as an argument and returns the result of multiplying `x` by itself, which is the square of the input. This method is a simple utility for performing a square operation in a mathematical context.'},
 {'code': '\nclass Animal {\npublic void speak() {\nSystem.out.println("Animal sound");\n}\n}\n ',
  'methodname': 'speak','classname':'Animal','method_signature':'speak()',
  'intent': ' The `speak` method in the `Animal` class in the provided C++ or Java source code is a public method that prints the message "Animal sound" to the console. This method provides a simple way to make an abstract representation of an animal produce a sound, although it does not provide any specific sound for any particular animal species.'},
 {'code': ' \nclass Dog extends Animal {\n@Override\npublic void speak() {\nSystem.out.println("Woof!");\n}\n}\n',
  'methodname': 'speak','classname':'Animal, Dog','method_signature':'speak()',
  'intent': ' This is a Java method that overrides the speak() method in the Dog class, which is a subclass of the Animal class. When this method is called, it prints the string "Woof!" to the console, representing the sound that a dog makes.'},
 {'code': '\n public <T extends Number> double add(T a, T b) {\nreturn a.doubleValue() + b.doubleValue();\n}\n\n ',
  'methodname': 'add','classname':'Number','method_signature':'add(T , T )',
  'intent': ' This method, with a generic type T that extends the Number class, is implemented in either Java or a compatible language. Its purpose is to add together two numeric values of the same type, returning the result as a double precision floating-point number.'},
 {'code': '\nclass FileHandler {\n// No automatic destructor; must use explicit method\npublic void close() {\nSystem.out.println("Cleaning up resources.");\n}\n@Override\nprotected void finalize() throws Throwable {\n// Called by GC, but not reliable\nclose();\n}\n}\n  ',
  'methodname': 'close','classname':'FileHandler','method_signature':'close()',
  'intent': ' The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup. The `finalize()` method is overridden to call `close()`, but it is not reliable as it is dependent on the garbage collector.'},
 {'code': '\nclass Complex {\ndouble real, imag;\nComplex(double r, double i) { real = r; imag = i; }\npublic Complex add(Complex other) {\nreturn new Complex(real + other.real, imag + other.imag);\n}\n}\n ',
  'methodname': '','classname':'Complex','method_signature':'add(Complex )',
  'intent': ' This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to represent complex numbers. The `add` method takes a `Complex` number as an argument and returns a new `Complex` number representing the sum of the current and argument complex numbers.'},
  {'code': """public static double divide(double numerator, double denominator) {
        if (denominator == 0) {
            throw new ArithmeticException("Cannot divide by zero");
        }
        return numerator / denominator;
    }
""",
  'methodname': 'divide','classname':'','method_signature':' divide(double , double )','intent':'The function takes tow numbers and divides it'},
  {'code': """public class MathUtils {

    // Method to return the square root of a number
    public static double squareRoot(double number) {
        if (number < 0) {
            throw new IllegalArgumentException("Cannot compute square root of a negative number.");
        }
        return Math.sqrt(number);
    }
""",
  'methodname': 'squareRoot','classname':'MathUtils','method_signature':'squareRoot(double )','intent':'The function takes a number and gives the square root of it'}]

In [5]:
def create_documents_from_intents(chunk_list):
    list_of_documents = []
    for chunk in chunk_list:
        document = Document(
            page_content=f"'intent': {chunk['intent']},methods: {chunk['methodname']},classname: {chunk['classname']},method_signature:{chunk['method_signature']}",
            metadata = {'methods': chunk['methodname'],'classname': chunk['classname'],'code':chunk['code']}
        )
        list_of_documents.append(document)
    return list_of_documents
only_intent_document = create_documents_from_intents(code_list)

In [6]:
embedding = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-base",
            model_kwargs={"device": "cpu"})
vector_store1 = FAISS(
            embedding_function=embedding,
            index=faiss.IndexFlatL2(len(embedding.embed_query("hello world"))),
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
        )
vector_store1.add_documents(only_intent_document)

C:\Users\soham\AppData\Local\Temp\ipykernel_6476\4282650126.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings(
c:\c\Users\Hari\VISTAR_PROJECT\SM7to8_api_knowledge_mapping\myenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


['59437caf-d04d-4914-a50e-2e2b6ce5ea98',
 'daf15184-84b5-41fe-9779-4d89d0eaf2c6',
 '2f8fab05-20a2-445f-8d40-a466d3b08bd1',
 'e969ca62-7585-4d53-9dde-c58016a42878',
 '3298c8c9-4a33-4aa4-b785-ed1a55646dfb',
 '0d0e73d0-9fce-4287-a208-e0342901988d',
 'c1d75da0-ac57-47f2-a8df-99ba70249445',
 '3e3241f2-f2fa-4e67-ab28-0d6341f09426',
 'fb655df3-0bbf-428b-bceb-80af0a1e278d',
 'f173dcca-84dd-46ce-b9cd-04cbdbeb5d8c',
 'bd251427-b9fe-4067-bc57-0fd7eea6ae75',
 '3191f745-2df2-47ac-917e-f619163e31a8',
 'e41dc427-1fc3-4c13-b18b-9924d2b67bb5',
 '329f90fd-9a8e-43e4-86cf-e62c30e2afb3',
 'a022d09e-1fa9-477d-a39e-b10f0adef6d1',
 'd5899c41-9a41-4dda-aa69-ce986c5c2cfd']

In [82]:
# Example Mean:
code = """
 int mean(int a, int b) {
    return (a + b) / 2.0;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)

 Intent: Calculate the average of two integers, Operation: By adding the two integers, converting the result to a floating point number, and then dividing by 2.0.
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double )
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )
* 'intent': The function takes a number and gives the squar

In [24]:
# Example Root Mean:
code = """
 int rootMeanSquare(int a, int b) {
    return std::sqrt((a * a + b * b) / 2.0);
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


 Intent: Calculates the root mean square of two integers, 
Operation: By finding the average of the squares of the two inputs and then calculating its square root.
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double )
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: ,method_signature:multiply(int , int )
* 'intent':  This is a Java method nam

In [8]:
# Example geomentric Mean:
code = """
 double geometricMean(double a, double b) {
    return std::sqrt(a * b);
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


 Intent: Calculate the geometric mean of two numbers, Operation: Performs the operation by multiplying the two numbers and then taking the square root of the product.
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: ,method_signature:multiply(int , int )
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double )
* 'intent': The function takes tow 

In [9]:
# Example harmonic mean
code = """
 double harmonicMean(double a, double b) {
    return (2.0 * a * b) / (a + b);
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


 Intent: Calculate the harmonic mean of two numbers, Operation: Performs the calculation by averaging the reciprocals of the given numbers and then finding the product of those reciprocals and 2, then dividing by the sum of the numbers.
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double )
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )
* 'intent':  This is a simple method in Java that takes two integer paramete

In [10]:
# Example variance
code = """
double variance(double a, double b) {
    double mean = (a + b) / 2.0;
    return ((a - mean) * (a - mean) + (b - mean) * (b - mean)) / 2.0;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


 Intent: Calculate the variance of two numbers, Operation: By finding the mean of the two numbers, then calculating the difference of each number from the mean, squaring those differences, averaging the squared differences, and returning the result.
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double )
* 'intent':  This is a Java method named "add" that takes two i

In [11]:
# Example Standard Deviation
code = """
 double standardDeviation(double a, double b) {
    double mean = (a + b) / 2.0;
    double variance = ((a - mean) * (a - mean) + (b - mean) * (b - mean)) / 2.0;
    return std::sqrt(variance);
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


 Intent: Calculate the standard deviation of two numbers, operation: By first finding the mean, then calculating the variance, and finally returning the square root of the variance.
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double )
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other wo

In [25]:
# Example Default argument
code = """
 int add(int a, int b = 10) {
return a + b;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


 Intent: This method performs an addition operation, returning the sum of two integers.
Operation: It takes an integer 'a' as a parameter and an optional integer 'b' with a default value of 10, and returns the sum of the two numbers by adding them together.
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: ,method_signature:multiply(int , int )
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`. The method returns the maximum of the two input values by using a conditional operator. If `a` is greater than `b`, the method returns `a`, otherwise it returns `b`.,me

In [26]:
# Example Pass by reference

code = """
 void increment(int &x) {
x++;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


 Intent: Increment the value of a given integer, Operation: By using the postfix increment operator 'x++', which first returns the current value of x and then increments it.
* 'intent':  This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.,methods: increment,classname: MutableInt,method_signature:increment(MutableInt )
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: ,method_signature:multiply(int , int )
* 'intent': The function

In [14]:
# Example Reference

code = """
 void increment(int &x) {
x++;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


 Intent: Increments the value of a given integer, Operation: Performs the increment operation directly on the integer reference provided as an argument.
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: ,method_signature:multiply(int , int )
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )
* 'intent':  This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.,methods: increment,classname: MutableInt,method_signature:increment(MutableInt )
* 'intent': The function takes a number and g

In [27]:
# example File Handler example
code = """
class FileHandler {
public:
~FileHandler() {
// Automatically called when object goes out of scope
std::cout << "Cleaning up resources." << std::endl;
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
# my_dict = json.loads(matches[0])



* 'intent':  The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup. The `finalize()` method is overridden to call `close()`, but it is not reliable as it is dependent on the garbage collector.,methods: close,classname: FileHandler,method_signature:close()
* 'intent':  This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.,methods: printMessage,classname: ,method_signature:printMessage()
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )
{'id': 'gen-1756766237-Wu649IjNQlnxUaDB03MC', 'provider': 'DeepInfra',

In [74]:
# example 7
code = """
class Complex {
public:
double real, imag;
Complex(double r, double i): real(r), imag(i) {}
Complex operator+(const Complex& other) {
return Complex(real + other.real, imag + other.imag);
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])


* 'intent':  This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to represent complex numbers. The `add` method takes a `Complex` number as an argument and returns a new `Complex` number representing the sum of the current and argument complex numbers.,methods: ,classname: Complex,method_signature:add(Complex )
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double 

# API_MAPPING_FEWSHOT_CHAIN_OF_THOUGHT

In [2]:
import os
import requests
import json

def create_api_mapping(cpp_code, java_methods_info):
      
    # Prepare the prompt
    prompt = f"""You are an expert software engineer with deep expertise in both C++ and Java.

    ### Guidelines

Lets think step by step:

Step 1: Analyze the behavior and intent of the given C++ method considering its operation like(arithmetic, conditional, looping) and Language Feature Mappings ie all C++ language-level constructs that do not have direct equivalents in Java and must be expressed via one or more explicit Java methods or design patterns like (detect: operator overloading, default arguments, pass-by-reference, pointers, destructors, copy/move semantics, templates, implicit conversions, RAII)
Step 2: From the list of java methods provided which has the information about the intent, classname, methodname and method signature.
        For each candidate Java method, check if it alone can replicate the behavior of the C++ method, if yes add the signature of that java method to the `java_methods_used` list.
        And check all possible combinations of two or more methods. Include all the combinations of methods that fully replicate the behavior of C++ code as method signatures in `java_methods_used` list
        The list should be the list of signature of these java methods.
Step 3: Form a relationship between the methods from list of methods created from previous steps 
        - if there is only one function, then flow - signature of that one and only function and realtionship - signature of that one and only function
        - if there exist a group of function then check how they replicate the exact C++ behaviour like some function together and some functions alone. 
            - if we need all the methods combined together to replicate the behaviour then relationship - signature of method 1 AND signature of method 2 AND signature of method 3 AND signature of method 4
            - if all the methods can replicate alone then relationship -  signature of method 1 OR signature of method 2 OR signature of method 3 OR signature of method 4
            - if some of them replicate alone and some combined together to replicate the exact C++ behaviour then relationship - (signature of method 1) OR (signature of methods 2) OR (signature of method 3 AND signature of method 4)
Step 4: From the given c++ code we should check for Language Feature Mappings like overloading and it exists then the mapping is one to many whether it 
        contains one java method or many java method.
        - if the Language Feature Mappings like overloading does not exists on C++ code, and number of methods present in the list is one then mapping is one to one. 
        - else then mapping is one to many.
Step 5: If no combination can fully replicate the behavior, say "No exact match. skipped


### Example 1:
C++ method:
int mean(int a, int b) {{
    return (a + b) / 2.0;
}}

Candidate Java methods:
- add(int a, int b)
- divide(double a, double b)
- multiply(double a, double b)
- squareRoot(double x)

Reasoning:
1. This computes the average of two integers.
2. We need to add the two integers, then divide by 2.0.
3. Relationship is AND as we need both the methods
4. Mapping: one-to-many because two Java methods are used and no language feature mapping.

JSON:
{{
  "match": true,
  "java_methods_used": ["add(int, int)", "divide(double, double)"],
  "api_flow": ["add(int, int) -> divide(double, double)"],
  "Relationship": "add(int, int) AND divide(double, double)"
  "Mapping": "one-to-many",
  "Intent": "Adds two integers and divides the sum by 2.0 to find the mean."
}}

---

### Example 2:
C++ method:
Complex Complex::operator+(const Complex& other) {{
    return Complex(real + other.real, imag + other.imag);
}}

Candidate Java methods:
- Complex.add(Complex other)
- add(int a, int b)
- multiply(double a, double b)

Reasoning:
1. This adds two complex numbers using operator overloading.
2. The Java equivalent is Complex.add(Complex other).
3. Relationship is Complex.add(Complex other) as we have one method to replicate the behaviour
4. Mapping: one-to-many because operator overloading is a language feature mapping in C++ translates to a method in Java.

JSON:
{{
  "match": true,
  "java_methods_used": ["Complex.add(Complex)"],
  "api_flow": ["Complex.add(Complex)"],
  "Relationship": "Complex.add(Complex other)"
  "Mapping": "one-to-many",
  "Intent": "Adds two complex numbers just like operator+."
}}

---
### Example 3:
C++ method:
int mean(int a, int b) {{
    return (a + b) / 2.0;
}}

Candidate Java methods:
- add(int a, int b)
- divide(double a, double b)
- multiply(double a, double b)
- squareRoot(double x)
- mean(double a , double b)

Reasoning:
1. This computes the average of two integers.
2. We need to add the two integers, then divide by 2.0, this one way and other way is the actual mean function which calulates the mean of it.
3. Relationship is both AND for add and divide, as we need both the methods and OR for those combination of methods with mean method
3. Mapping: one-to-many because two Java methods are used and no language feature mapping.

JSON:
{{
  "match": true,
  "java_methods_used": ["add(int, int)", "divide(double, double)", "mean(double, double)"],
  "api_flow": ["add(int, int) -> divide(double, double) |  mean(double, double)"],
  "Relationship": "(add(int, int) AND divide(double, double)) OR (mean(double, double))"
  "Mapping": "one-to-many",
  "Intent": "Adds two integers and divides the sum by 2.0 to find the mean."
}}

---

### Now your task:
C++ method:
{cpp_code}

Candidate Java methods:
{java_methods_info}

Follow the same reasoning steps and return only the JSON result.
"""
    api_key = os.environ["API_KEY"]
    response = requests.post(
    url="https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    },
    data=json.dumps({
        "model": "qwen/qwen-2.5-coder-32b-instruct",
        "messages": [
            {"role": "system", "content": "You are a helpful expert software analyst."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens":1024,
        "temperature": 0.2,
        
    })
    )
    result = response.json()
    print(result)
    return result["choices"][0]["message"]["content"]

In [3]:
# Geomentric Mean with function:

code = """
 double geometricMean(double a, double b) {
    return std::sqrt(a * b);
}
"""
java_methods = ["* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )",
                "* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )",
                "* 'intent':  This is a Java method named 'add' that takes two integer parameters, 'a' and 'b'. The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )",
                "* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double)",
                "* 'intent': The function takes two numbers and gives geometric mean of it,methods: Geomentric_Mean,classname: ,method_signature: Geometric_Mean(double , double )"]
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


{'id': 'gen-1756975395-2u0olHqle30vP86QN82Z', 'provider': 'DeepInfra', 'model': 'qwen/qwen-2.5-coder-32b-instruct', 'object': 'chat.completion', 'created': 1756975396, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "match": true,\n  "java_methods_used": ["multiply(double, double)", "squareRoot(double)", "Geometric_Mean(double, double)"],\n  "api_flow": ["multiply(double, double) -> squareRoot(double) | Geometric_Mean(double, double)"],\n  "Relationship": "(multiply(double, double) AND squareRoot(double)) OR (Geometric_Mean(double, double))",\n  "Mapping": "one-to-many",\n  "Intent": "Calculates the geometric mean of two double precision numbers."\n}\n```', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 1579, 'completion_tokens': 116, 'total_tokens': 1695, 'prompt_tokens_details': None}}
The methods and mapping are, ['{\n  "match": true,\n  "java_methods_use

In [4]:
# Example harmonic mean with function
code = """
 double harmonicMean(double a, double b) {
    return (2.0 * a * b) / (a + b);
}
"""
java_methods = ["* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )",
                "* 'intent': The function takes two numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )",
                "* 'intent': The function takes two numbers and calculates harmonic mean of them,methods: harmonic_mean,classname: ,method_signature: harmonic_mean(double , double )"
                "* 'intent':  This is a Java method named 'add' that takes two integer parameters, 'a' and 'b'. The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )",
                "* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double)",]
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


{'id': 'gen-1756975401-h9YuXJOgub6ZeoVDfiQ5', 'provider': 'DeepInfra', 'model': 'qwen/qwen-2.5-coder-32b-instruct', 'object': 'chat.completion', 'created': 1756975401, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "match": true,\n  "java_methods_used": ["multiply(double, double)", "add(double, double)", "divide(double, double)", "harmonic_mean(double, double)"],\n  "api_flow": ["multiply(double, double) -> add(double, double) -> divide(double, double) | harmonic_mean(double, double)"],\n  "Relationship": "(multiply(double, double) AND add(double, double) AND divide(double, double)) OR (harmonic_mean(double, double))",\n  "Mapping": "one-to-many",\n  "Intent": "Calculates the harmonic mean of two double precision numbers."\n}\n```', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 1581, 'completion_tokens': 133, 'total_tokens': 1714, 'prompt_tokens_details': 

In [5]:
# Example harmonic mean with function
code = """
#include <iostream>
#include <string>
#include <cctype> // for toupper
int main() {
    std::string text = "Hello world\t!  ";
    std::cout << "Original string: " << text << std::endl;
    // Remove spaces and tabs
    std::string noSpaces = "";
    for (char c : text) {
        if (c != ' ' && c != '\t') {
            noSpaces += c;
        }
    }
    std::cout << "Without spaces: " << noSpaces << std::endl;
    // Convert to uppercase
    std::string upperCase = "";
    for (char c : noSpaces) {
        upperCase += std::toupper(c);
    }
    std::cout << "Uppercase: " << upperCase << std::endl;
    // Count characters
    int length = upperCase.length();
    std::cout << "Number of characters: " << length << std::endl;
    return 0;
}
"""
java_methods = ["* 'intent': The function takes a string and removes the spaces in the string,methods: removeSpaces,classname: ,method_signature:removeSpaces(String )",
                "* 'intent': The function takes a string and return the number of character of string it,methods: characterCount,classname: ,method_signature: characterCount(String )",
                "* 'intent': The function takes a string and changes the characters in the string to uppercase ,methods,methods: toUpperCase,classname: ,method_signature: toUpperCase(String )",
                "* 'intent': The function takes a string and prints string: divide,classname: ,method_signature: print(String )",
                "* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )",]
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


{'id': 'gen-1756975425-LVLguRGCzXxGVEHYywOr', 'provider': 'DeepInfra', 'model': 'qwen/qwen-2.5-coder-32b-instruct', 'object': 'chat.completion', 'created': 1756975425, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "match": true,\n  "java_methods_used": ["removeSpaces(String)", "toUpperCase(String)", "characterCount(String)"],\n  "api_flow": ["removeSpaces(String) -> toUpperCase(String) -> characterCount(String)"],\n  "Relationship": "removeSpaces(String) AND toUpperCase(String) AND characterCount(String)",\n  "Mapping": "one-to-many",\n  "Intent": "Removes spaces and tabs from a string, converts it to uppercase, and counts the number of characters."\n}\n```', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 1689, 'completion_tokens': 108, 'total_tokens': 1797, 'prompt_tokens_details': None}}
The methods and mapping are, ['{\n  "match": true,\n  "java_methods

In [10]:
def intent_generator(code):

        # Prepare the prompt
        prompt = f"""
        Analyze the following C++ or java source code and write a very very short intent and how it performs that intent of the method.
        just two sentences. it should also include how it performs as operation
        Excepted Output:
        "Intent: What is does, Operation: How it does"

        C++ or java Source Code:
        {code}
        """

        # Build the request payload for a chat-based model (like Mistral or Falcon via OpenRouter)
        data = {
            "model": "mistralai/mixtral-8x7b-instruct",  # or another model available on OpenRouter
            "messages": [
                {"role": "system", "content": "You are a helpful expert software analyst."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens": 1024,
            "temperature": 0.2
        }

        headers = {
            "Authorization": f"Bearer {os.environ['API_KEY']}",
            "Content-Type": "application/json"
        }

        response = requests.post('https://openrouter.ai/api/v1/chat/completions', json=data, headers=headers)

        if response.status_code == 200:
            result = response.json()
            # Extract generated text from the response structure
            description = result["choices"][0]["message"]["content"]
            return(description)
        else:
            return(f"Error: {response.status_code} - {response.text}")

In [5]:
code_list = [{'code': 'public void printMessage() {\nSystem.out.println("Hello, World!");\n}',
  'methodname': 'printMessage','classname':'','method_signature':'printMessage()',
  'intent': ' This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.'},
 {'code': 'public int maxVal(int a, int b) {\nreturn (a > b) ? a : b;\n}',
  'methodname': 'maxVal','classname':'','method_signature':'maxVal(int , int )',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`. The method returns the maximum of the two input values by using a conditional operator. If `a` is greater than `b`, the method returns `a`, otherwise it returns `b`.'},
 {'code': 'public int add(int a) {\nreturn add(a, 10);\n}',
  'methodname': 'add','classname':'','method_signature':'add(int )',
  'intent': ' This Java method, named "add", takes an integer "a" as an argument and calls the "add" method with arguments "a" and 10. The return type is an integer. The specific implementation of the add method is not provided, but it appears to perform some form of addition.'},
 {'code': 'public int add(int a, int b) {\nreturn a + b;\n}',
  'methodname': 'add','classname':'','method_signature':'add(int , int )',
  'intent': ' This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the \'+\' operator.'},
 {'code': 'public double multiply(double a, double b) {\nreturn a * b;\n}',
  'methodname': 'multiply','classname':'','method_signature':'multiply(double , double )',
  'intent': " This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase."},
 {'code': 'public int multiply(int a, int b) {\nreturn a * b;\n}',
  'methodname': 'multiply','classname':'','method_signature':'multiply(int , int )',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.'},
 {'code': 'public void increment(MutableInt x) {\nx.value++;\n}',
  'methodname': 'increment','classname':'MutableInt','method_signature':'increment(MutableInt )',
  'intent': ' This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.'},
 {'code': 'class MutableInt {\npublic int value;\npublic MutableInt(int v) { value = v; }\n}',
  'methodname': '','classname':'MutableInt','method_signature':'MutableInt(int )',
  'intent': ' The `MutableInt` class in C++ or Java is a simple container for an integer value. It provides a constructor to initialize the value and allows direct access to the value, making it mutable.'},
 {'code': 'class MathUtils {\npublic static int square(int x) {\nreturn x * x;\n}\n}',
  'methodname': 'square','classname':'MathUtils','method_signature':'square(int )',
  'intent': ' The `square` method in the `MathUtils` class takes an integer `x` as an argument and returns the result of multiplying `x` by itself, which is the square of the input. This method is a simple utility for performing a square operation in a mathematical context.'},
 {'code': '\nclass Animal {\npublic void speak() {\nSystem.out.println("Animal sound");\n}\n}\n ',
  'methodname': 'speak','classname':'Animal','method_signature':'speak()',
  'intent': ' The `speak` method in the `Animal` class in the provided C++ or Java source code is a public method that prints the message "Animal sound" to the console. This method provides a simple way to make an abstract representation of an animal produce a sound, although it does not provide any specific sound for any particular animal species.'},
 {'code': ' \nclass Dog extends Animal {\n@Override\npublic void speak() {\nSystem.out.println("Woof!");\n}\n}\n',
  'methodname': 'speak','classname':'Animal, Dog','method_signature':'speak()',
  'intent': ' This is a Java method that overrides the speak() method in the Dog class, which is a subclass of the Animal class. When this method is called, it prints the string "Woof!" to the console, representing the sound that a dog makes.'},
 {'code': '\n public <T extends Number> double add(T a, T b) {\nreturn a.doubleValue() + b.doubleValue();\n}\n\n ',
  'methodname': 'add','classname':'Number','method_signature':'add(T , T )',
  'intent': ' This method, with a generic type T that extends the Number class, is implemented in either Java or a compatible language. Its purpose is to add together two numeric values of the same type, returning the result as a double precision floating-point number.'},
 {'code': '\nclass FileHandler {\n// No automatic destructor; must use explicit method\npublic void close() {\nSystem.out.println("Cleaning up resources.");\n}\n@Override\nprotected void finalize() throws Throwable {\n// Called by GC, but not reliable\nclose();\n}\n}\n  ',
  'methodname': 'close','classname':'FileHandler','method_signature':'close()',
  'intent': ' The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup. The `finalize()` method is overridden to call `close()`, but it is not reliable as it is dependent on the garbage collector.'},
 {'code': '\nclass Complex {\ndouble real, imag;\nComplex(double r, double i) { real = r; imag = i; }\npublic Complex add(Complex other) {\nreturn new Complex(real + other.real, imag + other.imag);\n}\n}\n ',
  'methodname': '','classname':'Complex','method_signature':'add(Complex )',
  'intent': ' This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to represent complex numbers. The `add` method takes a `Complex` number as an argument and returns a new `Complex` number representing the sum of the current and argument complex numbers.'},
  {'code': """public static double divide(double numerator, double denominator) {
        if (denominator == 0) {
            throw new ArithmeticException("Cannot divide by zero");
        }
        return numerator / denominator;
    }
""",
  'methodname': 'divide','classname':'','method_signature':' divide(double , double )','intent':'The function takes tow numbers and divides it'},
  {'code': """public class MathUtils {

    // Method to return the square root of a number
    public static double squareRoot(double number) {
        if (number < 0) {
            throw new IllegalArgumentException("Cannot compute square root of a negative number.");
        }
        return Math.sqrt(number);
    }
""",
  'methodname': 'squareRoot','classname':'MathUtils','method_signature':'squareRoot(double )','intent':'The function takes a number and gives the square root of it'}]

In [6]:
def create_documents_from_intents(chunk_list):
    list_of_documents = []
    for chunk in chunk_list:
        document = Document(
            page_content=f"'intent': {chunk['intent']},methods: {chunk['methodname']},classname: {chunk['classname']},method_signature:{chunk['method_signature']}",
            metadata = {'methods': chunk['methodname'],'classname': chunk['classname'],'code':chunk['code']}
        )
        list_of_documents.append(document)
    return list_of_documents
only_intent_document = create_documents_from_intents(code_list)

In [7]:
embedding = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-base",
            model_kwargs={"device": "cpu"})
vector_store1 = FAISS(
            embedding_function=embedding,
            index=faiss.IndexFlatL2(len(embedding.embed_query("hello world"))),
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
        )
vector_store1.add_documents(only_intent_document)

C:\Users\soham\AppData\Local\Temp\ipykernel_12816\4282650126.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings(
c:\c\Users\Hari\VISTAR_PROJECT\SM7to8_api_knowledge_mapping\myenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


['f0e20283-ef53-4ba7-92f2-76f52a3aafab',
 '7f6a96b9-3649-4f45-8965-7bea2a509dc6',
 '75937e2f-7fbe-41a0-879e-6b445e092e84',
 '479082d9-d59a-4272-afa2-1fb451be05b9',
 'f382d32f-2afc-4748-b89e-01021d5a26bc',
 '22412fa9-e9b5-4ab5-8e5a-2a8ab61d68f6',
 '362dcd51-6cf7-4a35-921a-73025ecc15bb',
 'c442f0f3-9094-4fa7-80ca-0a62161d78b3',
 '3c841e70-d934-4297-8dc8-3ae778b7311c',
 '72f95728-a5db-4ee6-9cf7-3dd6e18a82b0',
 '0792485e-8b60-4a23-a673-bc2bc8b5afcc',
 '92328aac-85e5-4210-9e66-7b888fd22546',
 'f129258c-1e9e-45d3-a42a-12e8f3e18fee',
 'a4a2c5d1-45e5-4ef6-8729-4bbda0fee442',
 '00176fc1-2116-4c08-8e6d-9265adcf2149',
 'f51a0a62-23fa-4a21-bf53-71c1f1038571']

In [12]:
# Example Root Mean:
code = """
 int rootMeanSquare(int a, int b) {
    return std::sqrt((a * a + b * b) / 2.0);
}
"""
java_methods = ["* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )",
                "* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )",
                "* 'intent':  This is a Java method named 'add' that takes two integer parameters, 'a' and 'b'. The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )",
                "* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double)",]
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


{'id': 'gen-1756902495-m54R4DaORnSsb4HbWFht', 'provider': 'Nebius', 'model': 'qwen/qwen-2.5-coder-32b-instruct', 'object': 'chat.completion', 'created': 1756902495, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "match": true,\n  "java_methods_used": ["MathUtils.squareRoot(double)", "multiply(double, double)", "add(int, int)", "divide(double, double)"],\n  "api_flow": ["add(int, int) -> multiply(double, double) -> add(int, int) -> multiply(double, double) -> add(double, double) -> divide(double, double) -> MathUtils.squareRoot(double)"],\n  "Relationship": "add(int, int) AND multiply(double, double) AND add(double, double) AND divide(double, double) AND MathUtils.squareRoot(double)",\n  "Mapping": "one-to-many",\n  "Intent": "Calculates the root mean square of two integers."\n}\n```\n\n**Explanation:**\n1. **Behavior Analysis:** The C++ method calculates the Root Mean Square 

In [13]:
# Example geomentric Mean:
code = """
 double geometricMean(double a, double b) {
    return std::sqrt(a * b);
}
"""
java_methods = ["* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )",
                "* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )",
                "* 'intent':  This is a Java method named 'add' that takes two integer parameters, 'a' and 'b'. The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )",
                "* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double)",]
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


{'id': 'gen-1756902673-p2ltipcVyEm8uFikFTxk', 'provider': 'DeepInfra', 'model': 'qwen/qwen-2.5-coder-32b-instruct', 'object': 'chat.completion', 'created': 1756902673, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "match": true,\n  "java_methods_used": ["multiply(double, double)", "squareRoot(double)"],\n  "api_flow": ["multiply(double, double) -> squareRoot(double)"],\n  "Relationship": "multiply(double, double) AND squareRoot(double)",\n  "Mapping": "one-to-many",\n  "Intent": "Calculates the geometric mean of two double precision floating point numbers by multiplying them and then taking the square root of the product."\n}\n```', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 1541, 'completion_tokens': 101, 'total_tokens': 1642, 'prompt_tokens_details': None}}
The methods and mapping are, ['{\n  "match": true,\n  "java_methods_used": ["multiply(double, 

In [14]:
# Example harmonic mean
code = """
 double harmonicMean(double a, double b) {
    return (2.0 * a * b) / (a + b);
}
"""
java_methods = ["* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )",
                "* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )",
                "* 'intent':  This is a Java method named 'add' that takes two integer parameters, 'a' and 'b'. The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )",
                "* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double)",]
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


{'id': 'gen-1756902686-J8QmmVRolgmKhUl2OuwO', 'provider': 'Nebius', 'model': 'qwen/qwen-2.5-coder-32b-instruct', 'object': 'chat.completion', 'created': 1756902686, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "match": true,\n  "java_methods_used": ["multiply(double, double)", "add(double, double)", "divide(double, double)"],\n  "api_flow": ["multiply(double, double) -> multiply(double, double) -> add(double, double) -> divide(double, double)"],\n  "Relationship": "multiply(double, double) AND multiply(double, double) AND add(double, double) AND divide(double, double)",\n  "Mapping": "one-to-many",\n  "Intent": "Calculates the harmonic mean of two double precision numbers."\n}\n```\n\n**Note:** The candidate Java methods list did not include an `add(double, double)` method, but since the C++ method uses addition on `double` types, I assumed the existence of such a method fo

In [15]:
# Example variance
code = """
double variance(double a, double b) {
    double mean = (a + b) / 2.0;
    return ((a - mean) * (a - mean) + (b - mean) * (b - mean)) / 2.0;
}
"""
java_methods = ["* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )",
                "* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )",
                "* 'intent':  This is a Java method named 'add' that takes two integer parameters, 'a' and 'b'. The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )",
                "* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double)",
                "* 'intent': The function takes two numbers and subtracts it,methods: subtract,classname: ,method_signature: subtract(double , double )"]
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


{'id': 'gen-1756902931-PsY3iOEQpZSi4qz1FoCp', 'provider': 'DeepInfra', 'model': 'qwen/qwen-2.5-coder-32b-instruct', 'object': 'chat.completion', 'created': 1756902931, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "match": true,\n  "java_methods_used": ["add(double, double)", "divide(double, double)", "subtract(double, double)", "multiply(double, double)"],\n  "api_flow": ["add(double, double) -> divide(double, double) -> subtract(double, double) -> multiply(double, double) -> add(double, double) -> divide(double, double)"],\n  "Relationship": "add(double, double) AND divide(double, double) AND subtract(double, double) AND multiply(double, double) AND add(double, double) AND divide(double, double)",\n  "Mapping": "one-to-many",\n  "Intent": "Calculates the variance of two double numbers by finding the mean, then computing the squared differences from the mean, averaging thos

In [17]:
# Example Standard Deviation
code = """
 double standardDeviation(double a, double b) {
    double mean = (a + b) / 2.0;
    double variance = ((a - mean) * (a - mean) + (b - mean) * (b - mean)) / 2.0;
    return std::sqrt(variance);
}
"""
java_methods = ["* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )",
                "* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )",
                "* 'intent':  This is a Java method named 'add' that takes two integer parameters, 'a' and 'b'. The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )",
                "* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double)",
                "* 'intent': The function takes two numbers and subtracts it,methods: subtract,classname: ,method_signature: subtract(double , double )"]
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


{'id': 'gen-1756903458-7Sajn3kOX6DmsgSRrVld', 'provider': 'DeepInfra', 'model': 'qwen/qwen-2.5-coder-32b-instruct', 'object': 'chat.completion', 'created': 1756903458, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "match": true,\n  "java_methods_used": ["divide(double, double)", "multiply(double, double)", "subtract(double, double)", "MathUtils.squareRoot(double)"],\n  "api_flow": ["subtract(double, double) -> multiply(double, double) -> add(int, int) -> divide(double, double) -> MathUtils.squareRoot(double)"],\n  "Relationship": "subtract(double, double) AND multiply(double, double) AND add(int, int) AND divide(double, double) AND MathUtils.squareRoot(double)",\n  "Mapping": "one-to-many",\n  "Intent": "Calculates the standard deviation of two numbers by first finding the mean, then calculating the variance, and finally taking the square root of the variance."\n}\n```\n\n##

In [18]:
# Example Default argument
code = """
 int add(int a, int b = 10) {
return a + b;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


 Intent: This method adds two integers, with the second integer having a default value of 10.
Operation: It performs integer addition using the '+' operator, returning the sum of the two input integers.
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )
* 'intent':  This Java method, named "add", takes an integer "a" as an argument and calls the "add" method with arguments "a" and 10. The return type is an integer. The specific implementation of the add method is not provided, but it appears to perform some form of addition.,methods: add,classname: ,method_signature:add(int )
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: ,method_signature:multiply

In [19]:
# Example Pass by reference

code = """
 void increment(int &x) {
x++;
}
"""
intent = intent_generator(code)
print(intent)
java_methods = []

query = f"'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


 Intent: Increment the value of a given integer, Operation: By using the '++' operator to increase the value of the integer passed by reference.
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: ,method_signature:multiply(int , int )
* 'intent':  This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.,methods: increment,classname: MutableInt,method_signature:increment(MutableInt )
* 'intent': The function takes a number and gives the

In [20]:
# example File Handler example
code = """
class FileHandler {
public:
~FileHandler() {
// Automatically called when object goes out of scope
std::cout << "Cleaning up resources." << std::endl;
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
# my_dict = json.loads(matches[0])



* 'intent':  The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup. The `finalize()` method is overridden to call `close()`, but it is not reliable as it is dependent on the garbage collector.,methods: close,classname: FileHandler,method_signature:close()
* 'intent':  This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.,methods: printMessage,classname: ,method_signature:printMessage()
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )
{'id': 'gen-1756903515-6DuoKxoo4YdBZZvUlhRm', 'provider': 'DeepInfra',

In [21]:
# example 7
code = """
class Complex {
public:
double real, imag;
Complex(double r, double i): real(r), imag(i) {}
Complex operator+(const Complex& other) {
return Complex(real + other.real, imag + other.imag);
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])


* 'intent':  This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to represent complex numbers. The `add` method takes a `Complex` number as an argument and returns a new `Complex` number representing the sum of the current and argument complex numbers.,methods: ,classname: Complex,method_signature:add(Complex )
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double 

In [22]:
# Geomentric Mean with function:

code = """
 double geometricMean(double a, double b) {
    return std::sqrt(a * b);
}
"""
java_methods = ["* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )",
                "* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )",
                "* 'intent':  This is a Java method named 'add' that takes two integer parameters, 'a' and 'b'. The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )",
                "* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double)",
                "* 'intent': The function takes two numbers and gives geometric mean of it,methods: Geomentric_Mean,classname: ,method_signature: Geometric_Mean(double , double )"]
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


{'id': 'gen-1756903547-5rG1QHcJwD1sP2NZM6H4', 'provider': 'DeepInfra', 'model': 'qwen/qwen-2.5-coder-32b-instruct', 'object': 'chat.completion', 'created': 1756903547, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "match": true,\n  "java_methods_used": ["multiply(double, double)", "squareRoot(double)", "Geometric_Mean(double, double)"],\n  "api_flow": ["multiply(double, double) -> squareRoot(double) | Geometric_Mean(double, double)"],\n  "Relationship": "(multiply(double, double) AND squareRoot(double)) OR (Geometric_Mean(double, double))",\n  "Mapping": "one-to-many",\n  "Intent": "Calculates the geometric mean of two double precision numbers."\n}\n```', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 1579, 'completion_tokens': 116, 'total_tokens': 1695, 'prompt_tokens_details': None}}
The methods and mapping are, ['{\n  "match": true,\n  "java_methods_use

In [23]:
# Example harmonic mean with function
code = """
 double harmonicMean(double a, double b) {
    return (2.0 * a * b) / (a + b);
}
"""
java_methods = ["* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )",
                "* 'intent': The function takes two numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )",
                "* 'intent': The function takes two numbers and calculates harmonic mean of them,methods: harmonic_mean,classname: ,method_signature: harmonic_mean(double , double )"
                "* 'intent':  This is a Java method named 'add' that takes two integer parameters, 'a' and 'b'. The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )",
                "* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double)",]
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)


{'id': 'gen-1756903550-YLsSNesdcKIbMwx6r5nw', 'provider': 'Nebius', 'model': 'qwen/qwen-2.5-coder-32b-instruct', 'object': 'chat.completion', 'created': 1756903550, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "match": true,\n  "java_methods_used": ["multiply(double, double)", "add(double, double)", "divide(double, double)", "harmonic_mean(double, double)"],\n  "api_flow": ["multiply(double, double) -> add(double, double) -> divide(double, double) | harmonic_mean(double, double)"],\n  "Relationship": "(multiply(double, double) AND add(double, double) AND divide(double, double)) OR (harmonic_mean(double, double))",\n  "Mapping": "one-to-many",\n  "Intent": "Calculates the harmonic mean of two double precision numbers."\n}\n```\n\n**Explanation:**\n1. **Behavior Analysis:** The C++ method calculates the harmonic mean of two double numbers using arithmetic operations.\n2. **Ja

In [24]:
# example 7
code = """
int maxVal(int a, int b) {
return (a > b) ? a : b;
}

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])


* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`. The method returns the maximum of the two input values by using a conditional operator. If `a` is greater than `b`, the method returns `a`, otherwise it returns `b`.,methods: maxVal,classname: ,method_signature:maxVal(int , int )
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: ,method_signature:multiply(int , int )
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )
* 'intent': The function takes tow numbers and divides it,methods: divide,classname: ,method_signature: divide(double , double )
{'id': 'gen-1756903573-8lT4lhoQBeLFchXzZoKO', 'provider': 'DeepInfr

In [25]:
# example 7
code = """
int multiply(int a, int b) {
return a * b;
}};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])


* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: ,method_signature:multiply(int , int )
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase.,methods: multiply,classname: ,method_signature:multiply(double , double )
* 'intent':  This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the '+' operator.,methods: add,classname: ,method_signature:add(int , int )
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`. The method returns

In [26]:
# example 7
code = """
class MathUtils {
public:
static int square(int x) {
return x * x;
}
};

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])


* 'intent':  The `square` method in the `MathUtils` class takes an integer `x` as an argument and returns the result of multiplying `x` by itself, which is the square of the input. This method is a simple utility for performing a square operation in a mathematical context.,methods: square,classname: MathUtils,method_signature:square(int )
* 'intent': The function takes a number and gives the square root of it,methods: squareRoot,classname: MathUtils,method_signature:squareRoot(double )
* 'intent':  This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.,methods: multiply,classname: ,method_signature:multiply(int , int )
* 'intent':  This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for re

In [27]:
# example 7
code = """
class Animal {
public:
virtual void speak() {
std::cout << "Animal sound" << std::endl;
}
};
class Dog : public Animal {
public:
void speak() override {
std::cout << "Woof!" << std::endl;
}
}

"""
intent = intent_generator(code)
java_methods = []

query = f"code: {code},'intent': {intent}"
results = vector_store1.similarity_search(
    query,
    k=4,
)
for res in results:
    java_methods.append(res.page_content)
    print(f"* {res.page_content}")
response = create_api_mapping(code, java_methods)
pattern = r'\{[^}]*\}'

# Find all matches
matches = re.findall(pattern, response)
print("The methods and mapping are,",matches)
my_dict = json.loads(matches[0])


* 'intent':  This is a Java method that overrides the speak() method in the Dog class, which is a subclass of the Animal class. When this method is called, it prints the string "Woof!" to the console, representing the sound that a dog makes.,methods: speak,classname: Animal, Dog,method_signature:speak()
* 'intent':  The `speak` method in the `Animal` class in the provided C++ or Java source code is a public method that prints the message "Animal sound" to the console. This method provides a simple way to make an abstract representation of an animal produce a sound, although it does not provide any specific sound for any particular animal species.,methods: speak,classname: Animal,method_signature:speak()
* 'intent':  This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.,methods: printMessage,classname: ,method_signature:printMessage()
* 'intent': The function takes a number and 

# VECTOR STORE WITH HYBRID SEARCH

In [6]:
code_list = [{'code': 'public void printMessage() {\nSystem.out.println("Hello, World!");\n}',
  'methodname': 'printMessage','classname':'','method_signature':'printMessage()',
  'intent': ' This is a Java method named "printMessage" which prints the string "Hello, World!" to the console when called. Its purpose is to display a simple greeting message.'},
 {'code': 'public int maxVal(int a, int b) {\nreturn (a > b) ? a : b;\n}',
  'methodname': 'maxVal','classname':'','method_signature':'maxVal(int , int )',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`. The method returns the maximum of the two input values by using a conditional operator. If `a` is greater than `b`, the method returns `a`, otherwise it returns `b`.'},
 {'code': 'public int add(int a) {\nreturn add(a, 10);\n}',
  'methodname': 'add','classname':'','method_signature':'add(int )',
  'intent': ' This Java method, named "add", takes an integer "a" as an argument and calls the "add" method with arguments "a" and 10. The return type is an integer. The specific implementation of the add method is not provided, but it appears to perform some form of addition.'},
 {'code': 'public int add(int a, int b) {\nreturn a + b;\n}',
  'methodname': 'add','classname':'','method_signature':'add(int , int )',
  'intent': ' This is a Java method named "add" that takes two integer parameters, "a" and "b". The method returns the sum of these two integers, calculated by using the \'+\' operator.'},
 {'code': 'public double multiply(double a, double b) {\nreturn a * b;\n}',
  'methodname': 'multiply','classname':'','method_signature':'multiply(double , double )',
  'intent': " This is a simple method in Java that performs the operation of multiplying two given double precision floating point numbers, a and b, and returns the result of this multiplication. The method's purpose is to encapsulate the multiplication operation for reuse and to potentially provide abstraction in a larger codebase."},
 {'code': 'public int multiply(int a, int b) {\nreturn a * b;\n}',
  'methodname': 'multiply','classname':'','method_signature':'multiply(int , int )',
  'intent': ' This is a simple method in Java that takes two integer parameters, `a` and `b`, and returns their product. In other words, it performs integer multiplication.'},
 {'code': 'public void increment(MutableInt x) {\nx.value++;\n}',
  'methodname': 'increment','classname':'MutableInt','method_signature':'increment(MutableInt )',
  'intent': ' This method, `increment`, takes a single parameter `x` of type `MutableInt`, and increments the value of `x` by 1. This method is likely to be used in situations where the state of a mutable integer object needs to be updated.'},
 {'code': 'class MutableInt {\npublic int value;\npublic MutableInt(int v) { value = v; }\n}',
  'methodname': '','classname':'MutableInt','method_signature':'MutableInt(int )',
  'intent': ' The `MutableInt` class in C++ or Java is a simple container for an integer value. It provides a constructor to initialize the value and allows direct access to the value, making it mutable.'},
 {'code': 'class MathUtils {\npublic static int square(int x) {\nreturn x * x;\n}\n}',
  'methodname': 'square','classname':'MathUtils','method_signature':'square(int )',
  'intent': ' The `square` method in the `MathUtils` class takes an integer `x` as an argument and returns the result of multiplying `x` by itself, which is the square of the input. This method is a simple utility for performing a square operation in a mathematical context.'},
 {'code': '\nclass Animal {\npublic void speak() {\nSystem.out.println("Animal sound");\n}\n}\n ',
  'methodname': 'speak','classname':'Animal','method_signature':'speak()',
  'intent': ' The `speak` method in the `Animal` class in the provided C++ or Java source code is a public method that prints the message "Animal sound" to the console. This method provides a simple way to make an abstract representation of an animal produce a sound, although it does not provide any specific sound for any particular animal species.'},
 {'code': ' \nclass Dog extends Animal {\n@Override\npublic void speak() {\nSystem.out.println("Woof!");\n}\n}\n',
  'methodname': 'speak','classname':'Animal, Dog','method_signature':'speak()',
  'intent': ' This is a Java method that overrides the speak() method in the Dog class, which is a subclass of the Animal class. When this method is called, it prints the string "Woof!" to the console, representing the sound that a dog makes.'},
 {'code': '\n public <T extends Number> double add(T a, T b) {\nreturn a.doubleValue() + b.doubleValue();\n}\n\n ',
  'methodname': 'add','classname':'Number','method_signature':'add(T , T )',
  'intent': ' This method, with a generic type T that extends the Number class, is implemented in either Java or a compatible language. Its purpose is to add together two numeric values of the same type, returning the result as a double precision floating-point number.'},
 {'code': '\nclass FileHandler {\n// No automatic destructor; must use explicit method\npublic void close() {\nSystem.out.println("Cleaning up resources.");\n}\n@Override\nprotected void finalize() throws Throwable {\n// Called by GC, but not reliable\nclose();\n}\n}\n  ',
  'methodname': 'close','classname':'FileHandler','method_signature':'close()',
  'intent': ' The `close()` method in the `FileHandler` class is used to clean up resources. There is no automatic destructor in this class, so developers must explicitly call `close()` to ensure proper resource cleanup. The `finalize()` method is overridden to call `close()`, but it is not reliable as it is dependent on the garbage collector.'},
 {'code': '\nclass Complex {\ndouble real, imag;\nComplex(double r, double i) { real = r; imag = i; }\npublic Complex add(Complex other) {\nreturn new Complex(real + other.real, imag + other.imag);\n}\n}\n ',
  'methodname': '','classname':'Complex','method_signature':'add(Complex )',
  'intent': ' This code defines a simple `Complex` class in either C++ or Java (with minor modifications) to represent complex numbers. The `add` method takes a `Complex` number as an argument and returns a new `Complex` number representing the sum of the current and argument complex numbers.'},
  {'code': """public static double divide(double numerator, double denominator) {
        if (denominator == 0) {
            throw new ArithmeticException("Cannot divide by zero");
        }
        return numerator / denominator;
    }
""",
  'methodname': 'divide','classname':'','method_signature':' divide(double , double )','intent':'The function takes tow numbers and divides it'},
  {'code': """public class MathUtils {

    // Method to return the square root of a number
    public static double squareRoot(double number) {
        if (number < 0) {
            throw new IllegalArgumentException("Cannot compute square root of a negative number.");
        }
        return Math.sqrt(number);
    }
""",
  'methodname': 'squareRoot','classname':'MathUtils','method_signature':'squareRoot(double )','intent':'The function takes a number and gives the square root of it'}]

In [ ]:
def create_documents_from_intents(chunk_list):
    list_of_documents = []
    for chunk in chunk_list:
        document = Document(
            page_content=f"'intent': {chunk['intent']},methods: {chunk['methodname']},classname: {chunk['classname']},method_signature:{chunk['method_signature']}",
            metadata = {'methods': chunk['methodname'],'classname': chunk['classname'],'code':chunk['code']}
        )
        list_of_documents.append(document)
    return list_of_documents
only_intent_document = create_documents_from_intents(code_list)